In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum
 
spark = SparkSession.builder \
    .appName("Sales Partitioning by Territory") \
    .getOrCreate()
 
sales_df = spark.read.csv("sales.csv", header=True, inferSchema=True)
 
sales_by_territory = sales_df.groupBy("TERRITORY") \
    .agg(sum("SALES").alias("Total_Sales")) \
    .orderBy("TERRITORY")
 
sales_by_territory.show()
 
spark.stop()

+---------+------------------+
|TERRITORY|       Total_Sales|
+---------+------------------+
|     APAC| 746121.8300000002|
|     EMEA|4979272.4099999955|
|    Japan| 455173.2200000002|
|       NA| 3852061.390000001|
+---------+------------------+



In [3]:
from pyspark.sql import SparkSession
 
spark = SparkSession.builder \
    .appName("Distinct Countries Count") \
    .getOrCreate()
 
sales_df = spark.read.csv("Sales.csv", header=True, inferSchema=True)
 
distinct_countries_count = sales_df.select("COUNTRY").distinct().count()
 
print("Number of distinct countries:", distinct_countries_count)
 
spark.stop()

Number of distinct countries: 19


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max
 
spark = SparkSession.builder \
    .appName("Maximum Sales by Year") \
    .getOrCreate()
 
sales_df = spark.read.csv("Sales.csv", header=True, inferSchema=True)
 
max_sales_by_year = sales_df.groupBy("YEAR_ID") \
    .agg(max("SALES").alias("Max_Sales")) \
    .orderBy("YEAR_ID")
 
max_sales_by_year.show()
 
spark.stop()

+-------+---------+
|YEAR_ID|Max_Sales|
+-------+---------+
|   2003|  11279.2|
|   2004|  12536.5|
|   2005|  14082.8|
+-------+---------+



In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, min
 
spark = SparkSession.builder \
    .appName("Customers with Least Purchases by Year") \
    .getOrCreate()
 
sales_df = spark.read.csv("Sales.csv", header=True, inferSchema=True)
 
total_quantity_by_customer_by_year = sales_df.groupBy("YEAR_ID", "CUSTOMERNAME") \
    .agg(sum("QUANTITYORDERED").alias("Total_Quantity")) \
    .orderBy("YEAR_ID", "Total_Quantity")
 
min_quantity_by_year = total_quantity_by_customer_by_year.groupBy("YEAR_ID") \
    .agg(min("Total_Quantity").alias("Min_Quantity"))
 
customers_least_purchases_by_year = total_quantity_by_customer_by_year.alias("t1").join(
    min_quantity_by_year.alias("t2"),
    (total_quantity_by_customer_by_year["YEAR_ID"] == min_quantity_by_year["YEAR_ID"]) &
    (total_quantity_by_customer_by_year["Total_Quantity"] == min_quantity_by_year["Min_Quantity"])
).selectExpr(
    "t1.YEAR_ID as YEAR_ID",
    "t1.CUSTOMERNAME as CUSTOMERNAME",
    "t1.Total_Quantity as Total_Quantity"
)
 
customers_least_purchases_by_year.show()
 
spark.stop()

+-------+--------------------+--------------+
|YEAR_ID|        CUSTOMERNAME|Total_Quantity|
+-------+--------------------+--------------+
|   2003|        Royale Belge|            47|
|   2004|Dragon Souveniers...|            28|
|   2005|   Boards & Toys Co.|            35|
+-------+--------------------+--------------+



In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum
 
spark = SparkSession.builder \
    .appName("Sales Partitioned by Country, State, and City") \
    .getOrCreate()
 
sales_df = spark.read.csv("Sales.csv", header=True, inferSchema=True)
 
sales_partitioned = sales_df.groupBy("COUNTRY", "STATE", "CITY") \
    .agg(sum("SALES").alias("Total_Sales")) \
    .orderBy("COUNTRY", "STATE", "CITY")
 
sales_partitioned.show()
 
spark.stop()

+---------+----------+--------------+------------------+
|  COUNTRY|     STATE|          CITY|       Total_Sales|
+---------+----------+--------------+------------------+
|Australia|       NSW|     Chatswood|151570.98000000004|
|Australia|       NSW|  North Sydney|153996.13000000003|
|Australia|Queensland|South Brisbane| 59469.11999999999|
|Australia|  Victoria|  Glen Waverly| 64591.46000000001|
|Australia|  Victoria|     Melbourne|200995.40999999997|
|  Austria|      NULL|          Graz|52263.899999999994|
|  Austria|      NULL|      Salzburg|         149798.63|
|  Belgium|      NULL|     Bruxelles|          74972.52|
|  Belgium|      NULL|     Charleroi|           33440.1|
|   Canada|        BC|     Tsawassen| 74634.84999999999|
|   Canada|        BC|     Vancouver|          75238.92|
|   Canada|    Quebec|      Montreal|          74204.79|
|  Denmark|      NULL|       Aaarhus|100595.54999999999|
|  Denmark|      NULL|     Kobenhavn|          145041.6|
|  Finland|      NULL|         